<a href="https://colab.research.google.com/github/MPGarg/Assignment-11/blob/main/Assignment11_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/MPGarg/Assignment-11.git

Cloning into 'Assignment-11'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 52 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), 1.94 MiB | 1.60 MiB/s, done.


In [3]:
ls

Assignment-11/  gdrive/  sample_data/


In [4]:
# Loading vocab & training file
!cp -r '/content/gdrive/MyDrive/Colab Notebooks/Assignment11/vocab.txt' /content/
!cp -r '/content/gdrive/MyDrive/Colab Notebooks/Assignment11/training.txt' /content/

In [5]:
ls

Assignment-11/  gdrive/  sample_data/  training.txt  vocab.txt


In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [17]:
# =============================================================================
# Libs
# =============================================================================
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re
#from random import *

# =============================================================================
# Transformer
# =============================================================================
def attention(q, k, v, mask = None, dropout = None):
    scores = q.matmul(k.transpose(-2, -1))
    scores /= math.sqrt(q.shape[-1])
    
    #mask
    scores = scores if mask is None else scores.masked_fill(mask == 0, -1e3)
    
    scores = F.softmax(scores, dim = -1)
    scores = dropout(scores) if dropout is not None else scores
    output = scores.matmul(v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, out_dim, dropout=0.1):
        super().__init__()
        
#        self.q_linear = nn.Linear(out_dim, out_dim)
#        self.k_linear = nn.Linear(out_dim, out_dim)
#        self.v_linear = nn.Linear(out_dim, out_dim)
        self.linear = nn.Linear(out_dim, out_dim*3)

        self.n_heads = n_heads
        self.out_dim = out_dim
        self.out_dim_per_head = out_dim // n_heads
        self.out = nn.Linear(out_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
    
    def split_heads(self, t):
        return t.reshape(t.shape[0], -1, self.n_heads, self.out_dim_per_head)
    
    def forward(self, x, y=None, mask=None):
        #in decoder, y comes from encoder. In encoder, y=x
        y = x if y is None else y
        
        qkv = self.linear(x) # BS * SEQ_LEN * (3*EMBED_SIZE_L)
        q = qkv[:, :, :self.out_dim] # BS * SEQ_LEN * EMBED_SIZE_L
        k = qkv[:, :, self.out_dim:self.out_dim*2] # BS * SEQ_LEN * EMBED_SIZE_L
        v = qkv[:, :, self.out_dim*2:] # BS * SEQ_LEN * EMBED_SIZE_L
        
        #break into n_heads
        q, k, v = [self.split_heads(t) for t in (q,k,v)]  # BS * SEQ_LEN * HEAD * EMBED_SIZE_P_HEAD
        q, k, v = [t.transpose(1,2) for t in (q,k,v)]  # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        
        #n_heads => attention => merge the heads => mix information
        scores = attention(q, k, v, mask, self.dropout) # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        scores = scores.transpose(1,2).contiguous().view(scores.shape[0], -1, self.out_dim) # BS * SEQ_LEN * EMBED_SIZE_L
        out = self.out(scores)  # BS * SEQ_LEN * EMBED_SIZE
        
        return out

class FeedForward(nn.Module):
    def __init__(self, inp_dim, inner_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(inp_dim, inner_dim)
        self.linear2 = nn.Linear(inner_dim, inp_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        #inp => inner => relu => dropout => inner => inp
        return self.linear2(self.dropout(F.relu(self.linear1(x)))) 

class EncoderLayer(nn.Module):
    def __init__(self, n_heads, inner_transformer_size, inner_ff_size, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, inner_transformer_size, dropout)
        self.ff = FeedForward(inner_transformer_size, inner_ff_size, dropout)
        self.norm1 = nn.LayerNorm(inner_transformer_size)
        self.norm2 = nn.LayerNorm(inner_transformer_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        x2 = self.norm1(x)
        x = x + self.dropout1(self.mha(x2, mask=mask))
        x2 = self.norm2(x)
        x = x + self.dropout2(self.ff(x2))
        return x

class Transformer(nn.Module):
    def __init__(self, n_code, n_heads, embed_size, inner_ff_size, n_embeddings, seq_len, dropout=.1):
        super().__init__()
        
        #model input
        self.embeddings = nn.Embedding(n_embeddings, embed_size)
        self.pe = PositionalEmbedding(embed_size, seq_len)
        
        #backbone
        encoders = []
        for i in range(n_code):
            encoders += [EncoderLayer(n_heads, embed_size, inner_ff_size, dropout)]
        self.encoders = nn.ModuleList(encoders)
        
        #language model
        self.norm = nn.LayerNorm(embed_size)
        self.linear = nn.Linear(embed_size, n_embeddings, bias=False)
                
    
    def forward(self, x):
        x = self.embeddings(x)
        x = x + self.pe(x)
        for encoder in self.encoders:
            x = encoder(x)
        x = self.norm(x)
        x = self.linear(x)
        return x

# Positional Embedding
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        pe.requires_grad = False
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return self.pe[:,:x.size(1)] #x.size(1) = seq_len
    
# =============================================================================
# Dataset
# =============================================================================
class SentencesDataset(Dataset):
    #Init dataset
    def __init__(self, sentences, vocab, seq_len):
        dataset = self
        
        dataset.sentences = sentences
        dataset.vocab = vocab + ['<ignore>', '<oov>', '<mask>']
        dataset.vocab_size = len(vocab)
        dataset.vocab = {e:i for i, e in enumerate(dataset.vocab)} 
        dataset.rvocab = {v:k for k,v in dataset.vocab.items()}
        dataset.seq_len = seq_len
        
        #special tags
        dataset.IGNORE_IDX = dataset.vocab['<ignore>'] #replacement tag for tokens to ignore
        dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>'] #replacement tag for unknown words
        dataset.MASK_IDX = dataset.vocab['<mask>'] #replacement tag for the masked word prediction task       
    
    #fetch data
    def __getitem__(self, index, p_random_mask=0.15):
        dataset = self
        
        #while we don't have enough word to fill the sentence for a batch
        s = []
        while len(s) < dataset.seq_len:
            s.extend(dataset.get_sentence_idx(index % len(dataset)))
            index += 1
        
        #ensure that the sequence is of length seq_len
        s = s[:dataset.seq_len]
        [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))] #PAD ok
        
        #apply random word 
        s1 = []
        for w in s:
          if random.random() < p_random_mask:
              prob = random.random()
              if prob < 0.8:
                  st = [dataset.MASK_IDX, w]
                  s1.append(st)                  
              elif prob < 0.9:
                  #any random word 10% of 15%
                  st = [dataset.vocab[dataset.rvocab[random.randrange(dataset.vocab_size)]], w]
                  s1.append(st)
              else:
                  #same word 10% of 15%
                  st = [w,w]
                  s1.append(st)
          else:
              st = [w, dataset.IGNORE_IDX]
              s1.append(st)
        
        #s = [((dataset.MASK_IDX, w) if random.random() < 0.8 elif (dataset.vocab[dataset.rvocab[randint(0, dataset.vocab_size - 1)]],w)) 
        #    if random.random() < p_random_mask else (w, dataset.IGNORE_IDX) for w in s]
        
        s = s1

        return {'input': torch.Tensor([w[0] for w in s]).long(),
                'target': torch.Tensor([w[1] for w in s]).long()}

    #return length
    def __len__(self):
        return len(self.sentences)

    #get words id
    def get_sentence_idx(self, index):
        dataset = self
        s = dataset.sentences[index]
        s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s] 
        return s

# =============================================================================
# Methods / Class
# =============================================================================
def get_batch(loader, loader_iter):
    try:
        batch = next(loader_iter)
    except StopIteration:
        loader_iter = iter(loader)
        batch = next(loader_iter)
    return batch, loader_iter



In [18]:
# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

initializing..


In [19]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = 'training.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = 'vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)


loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...


In [31]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 20
model.train()
batch_iter = iter(data_loader)
n_iteration = 2000
for it in range(n_iteration):
    
    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)
    
    #infer
    masked_input = batch['input']
    masked_target = batch['target']
    
    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)
    
    #compute the cross entropy loss 
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)
    
    #compute gradients
    loss.backward()
    
    #apply gradients
    optimizer.step()
    
    #print step
    if it % print_each == 0:
        print('it:', it, 
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))
    
    #reset gradients
    optimizer.zero_grad()
  

initializing model...
initializing optimizer and loss...
training...
it: 0  | loss 10.22  | Δw: 2.272
it: 20  | loss 8.44  | Δw: 0.652
it: 40  | loss 8.0  | Δw: 0.449
it: 60  | loss 7.67  | Δw: 0.394
it: 80  | loss 7.19  | Δw: 0.364
it: 100  | loss 6.92  | Δw: 0.385
it: 120  | loss 6.61  | Δw: 0.392
it: 140  | loss 6.29  | Δw: 0.418
it: 160  | loss 6.0  | Δw: 0.448
it: 180  | loss 5.7  | Δw: 0.407
it: 200  | loss 5.45  | Δw: 0.419
it: 220  | loss 5.39  | Δw: 0.472
it: 240  | loss 5.21  | Δw: 0.437
it: 260  | loss 5.06  | Δw: 0.468
it: 280  | loss 5.11  | Δw: 0.484
it: 300  | loss 4.89  | Δw: 0.484
it: 320  | loss 4.99  | Δw: 0.513
it: 340  | loss 4.9  | Δw: 0.545
it: 360  | loss 4.72  | Δw: 0.572
it: 380  | loss 4.78  | Δw: 0.653
it: 400  | loss 4.81  | Δw: 0.597
it: 420  | loss 4.73  | Δw: 0.658
it: 440  | loss 4.74  | Δw: 0.644
it: 460  | loss 4.81  | Δw: 0.653
it: 480  | loss 4.79  | Δw: 0.718
it: 500  | loss 4.74  | Δw: 0.8
it: 520  | loss 4.66  | Δw: 0.755
it: 540  | loss 4.79  | 

In [ ]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )


print('end')

In [32]:
# generate 10 input sentences taken all of same length to avoid padding
sentences_test = ['<mask> is the fourth planet from the sun and has a distinct',
             'The bright rust color <mask> is known for is due to iron-rich',
             'Mars also has the <mask> volcanoes in the solar system, Olympus Mons',
             'When Mars is closest to the <mask> its southern hemisphere is Cold',
             'A graphic showing brief timeline of important <mask> missions and the top',
             '<mask> storms are common on Mars. They can occur at any time',
             'NASA has two other orbiters working around the <mask> — the Mars',
             'The next two craft to successfully reach the Red <mask> were Mars',
             'We now know that <mask> is very cold and dry, with no',
             'Mars once had liquid <mask> on the surface and could have supported']

In [33]:
# convert the sentences to input tensors
inputs = torch.tensor([[dataset.vocab.get(word.lower(), 0)
                        for word in sentence.split()] for sentence in sentences_test])
inputs = inputs.to('cuda')

# process the inputs through the model
outputs = model(inputs)


# Prediction

In [34]:
# generate the output sentences
output_indices = torch.argmax(outputs, dim=2).tolist()
output_sentences = []
for indices in output_indices:
    words = [dataset.rvocab[index] for index in indices if index != dataset.vocab['<ignore>'] and not isinstance(index, list)]
    sentence = ' '.join(words)
    output_sentences.append(sentence)

# print the output sentences
for sentence in output_sentences:
    print(sentence)

<oov> is the <oov> <oov> from the <oov> and <oov> a <oov>
the <oov> <oov> <oov> <oov> is <oov> for is <oov> to ,
<oov> also <oov> the <oov> , in the , , <oov> <oov>
<oov> <oov> is <oov> to the <oov> <oov> <oov> , is <oov>
a , <oov> <oov> , of <oov> <oov> <oov> and the <oov>
<oov> <oov> are <oov> on , they <oov> , at <oov> time
, <oov> <oov> <oov> , <oov> , the of , the <oov>
the <oov> <oov> <oov> to <oov> <oov> the <oov> of were <oov>
we <oov> <oov> that <oov> is <oov> <oov> and , with <oov>
<oov> <oov> had <oov> <oov> on the , and . have the
